In [1]:
import sympy as sp
from lbmpy.stencils import LBStencil, Stencil
from lbmpy.moments import MOMENT_SYMBOLS, moment_matrix

In [2]:
d2q9 = LBStencil(Stencil.D2Q9)
Q = len(d2q9)
c = sp.Matrix(d2q9)
cs = sp.sqrt(sp.Rational(1,3))

x, y, z = MOMENT_SYMBOLS
one = sp.core.sympify(1)
c2 = x**2 + y**2
c4 = c2**2
moments = [
    one,
    x,
    y,
    3*c2 - 2,
    2*x**2 - c2,
    x*y,
    (3*c2-4)*x,
    (3*c2-4)*y,
    9*c4-15*c2+2
]
M = moment_matrix(moments, stencil=d2q9)

In [3]:
kappa = sp.symbols('kappa')
pb = sp.symbols('p_0')
rho = sp.symbols('rho',cls=sp.Function)(x,y,z)
ux, uy, uz = sp.symbols('u_x, u_y, u_z')
u = sp.Matrix([ux,uy])
Drho2 = sp.Matrix([ [ sp.Derivative(rho,a)*sp.Derivative(rho,b) for b in [x,y] ] for a in [x,y] ])
D2rho = sum([ sp.Derivative(rho,a,2) for a in [x,y] ])
G = kappa*Drho2

one12 = sp.Rational(1,12)
one18 = sp.Rational(1,18)
one72 = sp.Rational(1,72)

w = [ sp.Rational(4,9) ] + [ sp.Rational(1,9) ]*4 + [ sp.Rational(1,36) ]*4

ww =  [ [[0]*2]*2 ] \
    + [ [ [  -one18, 0 ], [ 0, 2*one18 ] ] ]*2 \
    + [ [ [ 2*one18, 0 ], [ 0,  -one18 ] ] ]*2 \
    + [ [ [ -one72, -one12 ], [ -one12, -one72 ] ],
        [ [ -one72,  one12 ], [  one12, -one72 ] ],
        [ [ -one72,  one12 ], [  one12, -one72 ] ],
        [ [ -one72, -one12 ], [ -one12, -one72 ] ] ]

def f(i):
    f = w[i]/cs**2*pb \
        + w[i]/cs**2*rho*c[i,:].dot(u) \
        + w[i]/(2*cs**4)*rho*u.dot((c[i,:].T@c[i,:]-cs**2*sp.eye(2))@u) \
        - w[i]/cs**2*(kappa*rho*D2rho) \
        + sum([ ww[i][j][j]/cs**2*G[j,j] for j in range(2) ]) \
        + sum([ ww[i][j][(j+1)%2]/cs**2*G[j,(j+1)%2] for j in range(1) ])
    return f

feq = sp.Matrix([ f(i) for i in range(Q) ])
feq[0] = rho - sum(feq[1:])

mf = sp.simplify(M@feq)

In [4]:
rho0 = sp.symbols('rho_0')
k = sp.symbols('k')
d = 6*(sp.symbols('c_s')**2+rho0*kappa*k**2 - cs**2)
gb,gs,gq,ge = sp.symbols('gamma_b, gamma_s, gamma_q, gamma_epsilon')
lb,ls,lq,le = sp.symbols('lambda_b, lambda_s, lambda_q, lambda_epsilon')
Omega = sp.zeros(Q,Q)
Omega[3,3] = lb
Omega[4,4] = ls
Omega[6,6] = lq
Omega[8,8] = le
Omega[5,5] = Omega[4,4]
Omega[7,7] = Omega[6,6]
Omega[3,0] = -lb*d
Omega[8,0] = le*d

In [5]:
kappa = sp.symbols('kappa')
pb = sp.symbols('p_0')
rho = sp.symbols('rho',cls=sp.Function)(x,y,z)
ux, uy, uz = sp.symbols('u_x, u_y, u_z')
kx, ky, kz = sp.symbols('k_x, k_y, k_z')
ks = {x:kx,y:ky,z:kz}
u = sp.Matrix([ux,uy])
Drho2 = sp.Matrix([ [ 0 for b in [x,y] ] for a in [x,y] ])
D2rho = -k**2*rho
G = kappa*Drho2

one12 = sp.Rational(1,12)
one18 = sp.Rational(1,18)
one72 = sp.Rational(1,72)

w = [ sp.Rational(4,9) ] + [ sp.Rational(1,9) ]*4 + [ sp.Rational(1,36) ]*4

ww =  [ [[0]*2]*2 ] \
    + [ [ [  -one18, 0 ], [ 0, 2*one18 ] ] ]*2 \
    + [ [ [ 2*one18, 0 ], [ 0,  -one18 ] ] ]*2 \
    + [ [ [ -one72, -one12 ], [ -one12, -one72 ] ],
        [ [ -one72,  one12 ], [  one12, -one72 ] ],
        [ [ -one72,  one12 ], [  one12, -one72 ] ],
        [ [ -one72, -one12 ], [ -one12, -one72 ] ] ]

def f(i):
    f = w[i]/cs**2*pb \
        + w[i]/cs**2*rho*c[i,:].dot(u) \
        + w[i]/(2*cs**4)*rho*u.dot((c[i,:].T@c[i,:]-cs**2*sp.eye(2))@u) \
        - w[i]/cs**2*(kappa*rho*D2rho) \
        + sum([ ww[i][j][j]/cs**2*G[j,j] for j in range(2) ]) \
        + sum([ ww[i][j][(j+1)%2]/cs**2*G[j,(j+1)%2] for j in range(1) ])
    return f

feq = sp.Matrix([ f(i) for i in range(Q) ])
feq[0] = rho - sum(feq[1:])

fbar = feq.subs({pb:rho*sp.symbols('c_s')**2}).subs({rho:rho0,ux:0,uy:0})
S = sp.symbols('S', cls=sp.Function)(k)/rho0*sp.diag(*fbar)

In [6]:
G = sp.simplify(M@S@M.T)
G

Matrix([
[                                       S(k),                                0,                                0, 2*(3*c_s**2 + 3*k**2*kappa*rho_0 - 1)*S(k),                                    0,                                  0,                                  0,                                  0, 2*(-3*c_s**2 - 3*k**2*kappa*rho_0 + 1)*S(k)],
[                                          0, (c_s**2 + k**2*kappa*rho_0)*S(k),                                0,                                          0,                                    0,                                  0,                                  0,                                  0,                                           0],
[                                          0,                                0, (c_s**2 + k**2*kappa*rho_0)*S(k),                                          0,                                    0,                                  0,                                  0,                        

In [7]:
Xi = sp.simplify(G-(sp.eye(Q,Q)+Omega)@G@(sp.eye(Q,Q)+Omega).T)
Xi

Matrix([
[0, 0, 0,                                                                                                                                                                                         0,                                                            0,                                                          0,                                                          0,                                                          0,                                                                                                                                                                                                                                            0],
[0, 0, 0,                                                                                                                                                                                         0,                                                            0,                                                       

In [8]:
d3q19 = LBStencil(Stencil.D3Q19)
Q = len(d3q19)
c = sp.Matrix(d3q19)
cs = sp.sqrt(sp.Rational(1,3))

x, y, z = MOMENT_SYMBOLS
one = sp.core.sympify(1)
c2 = x**2+y**2+z**2
c4 = c2**2

moments = [ 
    one,
    x,
    y,
    z,
    c2-one,
    3*x**2-c2,
    y**2-z**2,
    x*y,
    y*z,
    z*x,
    (3*c2-5)*x,
    (3*c2-5)*y,
    (3*c2-5)*z,
    (y**2-z**2)*x,
    (z**2-x**2)*y,
    (x**2-y**2)*z,
    3*c4-6*c2+one,
    (2*c2-3)*(3*x**2-c2),
    (2*c2-3)*(y**2-z**2)
]

M = moment_matrix(moments, stencil=d3q19)

In [9]:
k = sp.symbols('kappa')
Gamma = sp.symbols('Gamma_phi')
rho = sp.symbols('rho',cls=sp.Function)(x,y,z)
phi = sp.symbols('phi',cls=sp.Function)(x,y,z)
pb = sp.symbols('p_b')
mu = sp.symbols('mu_phi')
ux, uy, uz = sp.symbols('u_x, u_y, u_z')
u = sp.Matrix([ux,uy,uz])
D2rho = sum([ sp.Derivative(rho,a,2) for a in [x,y,z] ])
D2phi = sum([ sp.Derivative(phi,a,2) for a in [x,y,z] ])
Drho2 = sp.Matrix([ [ sp.Derivative(rho,a)*sp.Derivative(rho,b) for b in [x,y,z] ] for a in [x,y,z] ])
Dphi2 = sp.Matrix([ [ sp.Derivative(phi,a)*sp.Derivative(phi,b) for b in [x,y,z] ] for a in [x,y,z] ])
G = k*Drho2+k*Dphi2

five36 = sp.Rational(5,36)
one9 = sp.Rational(1,9)
one12 = sp.Rational(1,12)
one36 = sp.Rational(1,36)
one72 = sp.Rational(1,72)

w = [ sp.Rational(1,3) ] + [ sp.Rational(1,18) ]*6 + [ sp.Rational(1,36) ]*12

ww =  [ [[0]*3]*3 ] \
    + [ [ [ -one9, 0, 0], [0, five36, 0], [0, 0,  -one9] ] ]*2 \
    + [ [ [five36, 0, 0], [0,  -one9, 0], [0, 0,  -one9] ] ]*2 \
    + [ [ [ -one9, 0, 0], [0,  -one9, 0], [0, 0, five36] ] ]*2 \
    + [ [ [-one72, -one12, 0], [ -one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72,  one12, 0], [  one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72,  one12, 0], [  one12, -one72, 0], [ 0, 0, one36] ],
        [ [-one72, -one12, 0], [ -one12, -one72, 0], [ 0, 0, one36] ],
        [ [ one36, 0, 0], [ 0, -one72,  one12], [ 0,  one12, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72, -one12], [ 0, -one12, -one72] ],
        [ [-one72, 0, -one12], [ 0, one36, 0], [ -one12, 0, -one72] ],
        [ [-one72, 0,  one12], [ 0, one36, 0], [  one12, 0, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72, -one12], [ 0, -one12, -one72] ],
        [ [ one36, 0, 0], [ 0, -one72,  one12], [ 0,  one12, -one72] ],
        [ [-one72, 0,  one12], [ 0, one36, 0], [  one12, 0, -one72] ],
        [ [-one72, 0, -one12], [ 0, one36, 0], [ -one12, 0, -one72] ] ]

def f(i):
    f = w[i]/cs**2*pb \
        + w[i]/cs**2*rho*c[i,:].dot(u) \
        + w[i]/(2*cs**4)*rho*u.dot((c[i,:].T@c[i,:]-cs**2*sp.eye(3))@u) \
        - w[i]/cs**2*(k*rho*D2rho+k*phi*D2phi) \
        + sum([ ww[i][j][j]/cs**2*G[j,j] for j in range(3) ]) \
        + sum([ ww[i][j][(j+1)%3]/cs**2*G[j,(j+1)%3] for j in range(3) ])
    return f

feq = sp.Matrix([ f(i) for i in range(0,Q) ])
feq[0] = rho - sum(feq[1:])

def g(i):
    g = w[i]/cs**2*Gamma*mu \
        + w[i]/cs**2*phi*c[i,:].dot(u) \
        + w[i]/(2*cs**4)*phi*u.dot((c[i,:].T@c[i,:]-cs**2*sp.eye(3))@u)
    return g

geq = sp.Matrix([ g(i) for i in range(0,Q) ])
geq[0] = phi - sum(geq[1:])

In [10]:
mf = sp.simplify(M@feq)
mf

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 rho(x, y, z)],
[                                                                                                                                                                                                                                                                                                                             

In [11]:
mg = sp.simplify(M@geq)
mg

Matrix([
[                                                                                       phi(x, y, z)],
[                                                                                   u_x*phi(x, y, z)],
[                                                                                   u_y*phi(x, y, z)],
[                                                                                   u_z*phi(x, y, z)],
[3*Gamma_phi*mu_phi + u_x**2*phi(x, y, z) + u_y**2*phi(x, y, z) + u_z**2*phi(x, y, z) - phi(x, y, z)],
[                                                          (2*u_x**2 - u_y**2 - u_z**2)*phi(x, y, z)],
[                                                                     (u_y**2 - u_z**2)*phi(x, y, z)],
[                                                                               u_x*u_y*phi(x, y, z)],
[                                                                               u_y*u_z*phi(x, y, z)],
[                                                               